<a href="https://colab.research.google.com/github/spencergoldberg1/Data-Science/blob/develop/Emotion%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from torchsummary import summary

In [120]:
# 1. Mount Google Drive
drive.mount('/content/gdrive')

# 2. Load labeled images from folders
# data_dir = '/content/gdrive/MyDrive/Data1/antsbeesdataset'
data_dir = '/content/gdrive/MyDrive/DataSci/FacialRec'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [121]:
cd /content/gdrive/MyDrive/DataSci/FacialRec

/content/gdrive/MyDrive/DataSci/FacialRec


In [122]:
ls

alexnet_classification_emotions.pth  train/  valid/


In [123]:
# 3. Pre-process the data and create data loaders
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [124]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [125]:
# 4. Set up the AlexNet architecture
alexnet = models.alexnet(pretrained=True)

In [89]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [126]:

num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names)) # Change last layer
alexnet = alexnet.to(device) # Put on GPU

In [127]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [128]:
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [129]:
class_names

['happy', 'sad']

In [130]:
dataset_sizes

{'train': 140, 'valid': 80}

In [131]:
for inputs, labels in dataloaders["train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

In [132]:
inputs.shape

torch.Size([12, 3, 224, 224])

In [133]:
labels.shape

torch.Size([12])

In [134]:
labels

tensor([1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')

In [135]:
outputs = alexnet(inputs)

In [136]:
outputs.shape

torch.Size([12, 2])

In [137]:
outputs

tensor([[-1.0914,  0.5595],
        [ 0.4701,  0.4546],
        [-0.4703,  1.1518],
        [ 0.2162,  0.6941],
        [-0.1489,  0.8921],
        [-0.3233,  0.4163],
        [-0.5463,  0.6764],
        [-0.6402,  0.7813],
        [ 0.2344,  0.6165],
        [ 0.2450,  0.5777],
        [-0.2730,  0.6532],
        [ 0.4726,  0.7913]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [138]:
torch.max(outputs,1)[0]

tensor([0.5595, 0.4701, 1.1518, 0.6941, 0.8921, 0.4163, 0.6764, 0.7813, 0.6165,
        0.5777, 0.6532, 0.7913], device='cuda:0', grad_fn=<MaxBackward0>)

In [139]:
torch.max(outputs,1)[1]

tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [140]:
preds = torch.max(outputs, 1)[1]

In [141]:
preds

tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [142]:
labels

tensor([1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')

In [143]:
preds == labels

tensor([ True, False, False, False, False,  True,  True,  True,  True, False,
         True,  True], device='cuda:0')

In [144]:
labels.shape[0]

12

In [145]:
torch.sum(preds == labels)/labels.shape[0]

tensor(0.5833, device='cuda:0')

In [146]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Train the AlexNet model
alexnet.train()

num_epochs = 20

for epoch in range(num_epochs):
    print(epoch, " of ", num_epochs - 1)
    print('-' * 10)

    running_corrects = 0

    for inputs, labels in dataloaders["train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        

        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]



        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_corrects += torch.sum(preds == labels.data)

    print('Train Acc: {:.4f}'.format(running_corrects / dataset_sizes["train"]))




    # Evaluate the AlexNet model on Validation Data
    alexnet.eval()

    running_corrects = 0

    for inputs, labels in dataloaders["valid"]:
        inputs = inputs.to(device)
        labels = labels.to(device)


        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]


        running_corrects += torch.sum(preds == labels.data)

    acc_valid = running_corrects / dataset_sizes["valid"]
    print('Valid Acc: {:.4f}'.format(acc_valid))
    if acc_valid > 0.99:
        print("Done!")
        break


print('Training complete')

0  of  19
----------
Train Acc: 0.4929
Valid Acc: 0.6875
1  of  19
----------
Train Acc: 0.7071
Valid Acc: 0.5750
2  of  19
----------
Train Acc: 0.7714
Valid Acc: 0.5875
3  of  19
----------
Train Acc: 0.7286
Valid Acc: 0.5625
4  of  19
----------
Train Acc: 0.8429
Valid Acc: 0.5750
5  of  19
----------
Train Acc: 0.8500
Valid Acc: 0.5625
6  of  19
----------
Train Acc: 0.8286
Valid Acc: 0.5250
7  of  19
----------
Train Acc: 0.8429
Valid Acc: 0.5625
8  of  19
----------
Train Acc: 0.8929
Valid Acc: 0.5375
9  of  19
----------
Train Acc: 0.9071
Valid Acc: 0.5625
10  of  19
----------
Train Acc: 0.9143
Valid Acc: 0.5625
11  of  19
----------
Train Acc: 0.9214
Valid Acc: 0.5625
12  of  19
----------
Train Acc: 0.9143
Valid Acc: 0.5875
13  of  19
----------
Train Acc: 0.9000
Valid Acc: 0.5750
14  of  19
----------
Train Acc: 0.9071
Valid Acc: 0.5500
15  of  19
----------
Train Acc: 0.8714
Valid Acc: 0.6250
16  of  19
----------
Train Acc: 0.8929
Valid Acc: 0.6000
17  of  19
----------
Tr

In [147]:
pwd

'/content/gdrive/MyDrive/DataSci/FacialRec'

In [159]:
model_path = '/content/gdrive/MyDrive/DataSci/FacialRec/alexnet_classification_emotions.pth'

In [160]:
# Save the trained model
torch.save(alexnet.state_dict(), model_path)

In [161]:
ls

alexnet_classification_emotions.pth  train/  valid/


In [166]:
import requests
from PIL import Image
from io import BytesIO
import imageio as io
import matplotlib.pyplot as plt

# 1. Load the trained AlexNet model
def load_model(model_path):
    model = models.alexnet()
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

# 2. Define a function to load an image from a URL and preprocess it
def preprocess_image(url, transform):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = transform(img)
    return img_tensor.unsqueeze(0).to(device)

# 3. Perform inference using the loaded model
def predict_image_url(url, model):
    img_tensor = preprocess_image(url, data_transforms['valid'])
    output = model(img_tensor)
    pred = torch.max(output, 1)[1]
    return class_names[pred]

In [163]:
trained_model = load_model(model_path)

In [182]:
def plot(x):
    fig, ax = plt.subplots()
    im = ax.imshow(x, cmap = 'gray')
    ax.axis('off')
    fig.set_size_inches(18, 10)
    plt.show()

In [178]:
urls = ['https://personalexcellence.co/files/girl-smiling2.jpg', 'https://www.allprodad.com/wp-content/uploads/2021/03/05-12-21-happy-people.jpg', 'https://previews.123rf.com/images/bowie15/bowie151401/bowie15140100080/39843138-sad-man.jpg', 'https://st.depositphotos.com/2069237/2523/i/600/depositphotos_25235121-stock-photo-sad-crying-disappointed-funny-business.jpg', 'https://cdn.tinybuddha.com/wp-content/uploads/2016/01/Happy-Guy.jpg']

In [190]:
for image_url in urls:
  prediction = predict_image_url(image_url, trained_model)
  print("The predicted class for the input image is:", prediction)

The predicted class for the input image is: happy
The predicted class for the input image is: happy
The predicted class for the input image is: sad
The predicted class for the input image is: sad
The predicted class for the input image is: happy
